In [1]:
from pandas import DataFrame, read_csv
import random
import math

In [2]:
random.gauss(0, 0.5)

0.6618379768375289

In [3]:
paramsToStore = {"Time": [0,1,2,3,0,1,2,3,4,5],
                "TTC": [random.uniform(0, 10) for _ in range(10)],
                "DTO": [random.uniform(0, 20) for _ in range(10)],
                "JERK": [abs(random.gauss(0, 20)) for _ in range(10)],
                "Speed": [random.uniform(0, 15) for _ in range(10)],
                "AngularSpeedX": [random.gauss(0, 0.15) for _ in range(10)],
                "AngularSpeedY": [random.gauss(0, 0.15) for _ in range(10)],
                "AngularSpeedZ": [random.gauss(0, 0.1) for _ in range(10)],
                "COL": [0,0,0,1,0,0,0,0,1,0]}
df = DataFrame(paramsToStore)
# df.set_index("Time", inplace=True)
# df.to_csv(PATH+"/data/testGenerateData.csv", index=False)
df.head(10)

,Time,TTC,DTO,JERK,Speed,AngularSpeedX,AngularSpeedY,AngularSpeedZ,COL
0,0,3.346017,18.477581,18.308759,0.801559,0.158302,0.082666,0.077520,0
1,1,6.651468,1.082235,6.584725,9.982587,-0.025840,-0.165122,0.040654,0
2,2,7.112521,14.157006,16.642033,11.964243,-0.213264,-0.462122,0.084062,0
3,3,5.445207,12.209361,23.863087,12.066995,0.261510,0.047641,-0.010711,1
4,0,8.744643,6.858465,23.502004,8.026849,-0.168254,0.277582,-0.163770,0
5,1,7.305969,12.719076,11.261807,8.962356,0.169458,0.100560,0.151956,0
6,2,5.746609,7.311683,12.209406,11.630876,-0.022862,-0.039757,0.044711,0
7,3,7.424997,1.561131,1.957596,14.264494,0.006452,0.201171,0.162749,0
8,4,9.265533,0.112718,3.908058,13.143482,-0.040407,-0.181299,-0.108979,1
9,5,4.126269,16.794730,12.795391,12.185271,-0.191208,-0.188107,0.004098,0


In [4]:
l = [0]
l[-3:] = [1]*3
print(l)

[1, 1, 1]


In [5]:
def makeDataUsable(df: DataFrame, pastImportance: int=6, secBeforeCol: int=3) -> DataFrame:
    """
    Make a usable Dataframe from a csv file from generated data.\\
    Creates pastImportance of columns for each prediction feature.\\
    Registers the "col" feature before an actual collision as 1, up to secBeforeCol back in time.
    
    ### Params:
        * df: Dataframe, need to have the columns named "TTC", "DTO", "JERK" and "Speed"ArithmeticError
        * pastImportance: int, needs to be > 0
        * secBeforeCol: int, needs to be > 0
    ### Returns:
        * Dataframe
    """
    colsToUse = ["TTC", "DTO", "JERK", "Speed", "asX", "asY", "asZ"]
    columns = ["Time"]
    columns += [f"{c}{i}" for c in colsToUse for i in range(1, pastImportance+1)]
    columns += ["COL"]
    
    # colsToUse = list(df.columns)
    # columns = colsToUse[0]
    # columns += [f"{c}{i}" for i in range(1, pastImportance+1) for c in colsToUse[1:-1]]
    # columns = colsToUse[-1]
    
    dataDict = {col: [] for col in columns}

    for i, row in df.iterrows():
        if row["Time"] < pastImportance-1:
        # if row[colsToUse[0]] < pastImportance-1:   
            continue

        dataDict["Time"].append(row["Time"])
        # dataDict[colsToUse[0]].append(row[colsToUse[0]])
        for j, k in enumerate(range(i-pastImportance+1, i+1), start=1):
            for c in colsToUse:
            # for c in colsToUse[1:-1]:
                dataDict[f"{c}{j}"].append(df.iloc[k][c])

        dataDict[f"COL"].append(row["COL"])
        if row["COL"] == 1:
            available = int(row["Time"]-pastImportance+2)
            amount = int(secBeforeCol) if available >= secBeforeCol else available
            dataDict["COL"][-amount:] = [1]*amount

    # for key, val in dataDict.items():
    #     print(key, val, len(val))
    return DataFrame(dataDict)

# makeDataUsable(df, 3, 3)
makeDataUsable(read_csv("../../data/testGenerateData.csv"),4, 3)

,Time,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,...,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4,COL
0,3.0,0.000,50.000,50.000,50.000,0.0000,100.0000,100.0000,100.0000,0.000,...,-0.005,0.000,-0.000,-0.001,-0.022,0.000,0.004,-0.001,0.004,0.0
1,4.0,50.000,50.000,50.000,50.000,100.0000,100.0000,100.0000,100.0000,1.606,...,0.005,-0.000,-0.001,-0.022,-0.032,0.004,-0.001,0.004,-0.004,0.0
2,5.0,50.000,50.000,50.000,0.523,100.0000,100.0000,100.0000,6.1305,3.419,...,0.051,-0.001,-0.022,-0.032,0.235,-0.001,0.004,-0.004,-0.028,1.0
3,6.0,50.000,50.000,0.523,0.478,100.0000,100.0000,6.1305,6.3838,0.736,...,0.031,-0.022,-0.032,0.235,0.181,0.004,-0.004,-0.028,-0.019,1.0
4,7.0,50.000,0.523,0.478,50.000,100.0000,6.1305,6.3838,0.3667,6.073,...,-0.003,-0.032,0.235,0.181,0.000,-0.004,-0.028,-0.019,-0.016,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,56.0,50.000,50.000,50.000,5.362,100.0000,100.0000,100.0000,71.4008,2.549,...,0.005,0.443,-0.002,0.008,-0.001,-0.007,-0.018,0.007,0.006,0.0
112,57.0,50.000,50.000,5.362,5.479,100.0000,100.0000,71.4008,71.8032,0.695,...,-0.000,-0.002,0.008,-0.001,0.000,-0.018,0.007,0.006,-0.000,0.0
113,58.0,50.000,5.362,5.479,3.612,100.0000,71.4008,71.8032,67.9606,4.677,...,-0.000,0.008,-0.001,0.000,-0.002,0.007,0.006,-0.000,0.000,0.0
114,59.0,5.362,5.479,3.612,0.642,71.4008,71.8032,67.9606,12.5618,3.637,...,-0.009,-0.001,0.000,-0.002,0.001,0.006,-0.000,0.000,0.009,0.0


In [6]:
makeDataUsable(read_csv("../../data/testGenerateData.csv"), 5)

,Time,TTC1,TTC2,TTC3,TTC4,TTC5,DTO1,DTO2,DTO3,DTO4,...,asY2,asY3,asY4,asY5,asZ1,asZ2,asZ3,asZ4,asZ5,COL
0,4.0,0.000,50.000,50.000,50.000,50.000,0.0000,100.0000,100.0000,100.0000,...,-0.000,-0.001,-0.022,-0.032,0.000,0.004,-0.001,0.004,-0.004,0.0
1,5.0,50.000,50.000,50.000,50.000,0.523,100.0000,100.0000,100.0000,100.0000,...,-0.001,-0.022,-0.032,0.235,0.004,-0.001,0.004,-0.004,-0.028,1.0
2,6.0,50.000,50.000,50.000,0.523,0.478,100.0000,100.0000,100.0000,6.1305,...,-0.022,-0.032,0.235,0.181,-0.001,0.004,-0.004,-0.028,-0.019,1.0
3,7.0,50.000,50.000,0.523,0.478,50.000,100.0000,100.0000,6.1305,6.3838,...,-0.032,0.235,0.181,0.000,0.004,-0.004,-0.028,-0.019,-0.016,1.0
4,8.0,50.000,0.523,0.478,50.000,0.754,100.0000,6.1305,6.3838,0.3667,...,0.235,0.181,0.000,0.294,-0.004,-0.028,-0.019,-0.016,0.002,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,56.0,14.577,50.000,50.000,50.000,5.362,29.4314,100.0000,100.0000,100.0000,...,0.443,-0.002,0.008,-0.001,0.017,-0.007,-0.018,0.007,0.006,0.0
110,57.0,50.000,50.000,50.000,5.362,5.479,100.0000,100.0000,100.0000,71.4008,...,-0.002,0.008,-0.001,0.000,-0.007,-0.018,0.007,0.006,-0.000,0.0
111,58.0,50.000,50.000,5.362,5.479,3.612,100.0000,100.0000,71.4008,71.8032,...,0.008,-0.001,0.000,-0.002,-0.018,0.007,0.006,-0.000,0.000,0.0
112,59.0,50.000,5.362,5.479,3.612,0.642,100.0000,71.4008,71.8032,67.9606,...,-0.001,0.000,-0.002,0.001,0.007,0.006,-0.000,0.000,0.009,0.0


In [7]:
def calculateTTC(distance, speedA, accA, speedB, accB):
    """
    Calculate time to collision when both objects are moving and has an acceleration.
    ### Formulas
    d = vt + 0.5at^2\\
    0.5at^2 + vt + (-d) = 0
    --> t = (-v +- sqrt(v^2 - 4*0.5*a*d)) / 2 * 0.5 a
    """
    NO_COLLISION = 50
    relativeSpeed = speedA - speedB
    relativeAcceleration = accA - accB
    if relativeAcceleration == 0:
        if speedA <= speedB:
            return NO_COLLISION
        return distance / (speedA - speedB)
    
    num = relativeSpeed**2 - 4 * 0.5 * relativeAcceleration * (-distance)
    print(f"(-{relativeSpeed} +- sqrt({relativeSpeed}^2 - 4*0.5*{relativeAcceleration}*(-{distance}))) / 2 * 0.5 {relativeAcceleration}")
    if num >= 0:
        t_add = (-relativeSpeed + math.sqrt(num)) / (2*0.5*relativeAcceleration)
        t_sub = (-relativeSpeed - math.sqrt(num)) / (2*0.5*relativeAcceleration)
        if t_add > 0:
            return t_add
        elif t_sub > 0:
            return t_sub
    return NO_COLLISION # Not going to collide


def validateTTC(time, distance, speedA, accA, speedB, accB):
    def d(v, a, t):
        return v*t + 0.5 * a * t**2
    print(d(speedA, accA, time), distance)
    print(d(speedB, accB, time))
    if time >= 50:
        print("Not going to crash")
    if round(d(speedA, accA, time) - distance, 1) == round(d(speedB, accB, time), 1):
        print("Is correct")
    else:
        print("Something is wrong")


In [16]:
distance = 100

speedA = 15
accelerationA = 3

speedB = 10
accelerationB = 1

time = calculateTTC(distance, speedA, accelerationA, speedB, accelerationB)
print(f"Time to collision: {time} seconds")

validateTTC(time, distance, speedA, accelerationA, speedB, accelerationB)

(-5 +- sqrt(5^2 - 4*0.5*2*(-100))) / 2 * 0.5 2
Time to collision: 7.807764064044152 seconds
208.55823048033113 100
108.55823048033113
Is correct
